**Group - 22 **
* Arun M George(MT2016025) 
* Ashok Lathwal(MT2016028) 
* Athul Suresh(MT2016030) 
* Mithun Mathew(MT2016085) 
* Apoorva Bhalla(DT2017002)

***Clustering***

The primary aim of the sales dataset resides in using the current data to predict future sales. Here we've used clustering for feature engineering. We've tried to come up with relevant features to augement the given feature set so as to better predict future sales. 

In [42]:
import pandas as pd
import numpy as np
import xgboost as xgb
import uuid
from sklearn import cross_validation
from datetime import date, timedelta
from sklearn.cross_validation import KFold, train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from scipy.stats import pearsonr
import numpy as np
from sklearn.cluster import AgglomerativeClustering

In [43]:
print("Load the training, test and store data using pandas")
train = pd.read_csv("../input/train.csv")
test  = pd.read_csv("../input/test.csv")
store = pd.read_csv("../input/store.csv")
train.head()

Load the training, test and store data using pandas


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [44]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


**A pivot table is produced which shows the sales value in all the different store for each given date.
Before jumping into the actual pivot table creation, NA values are dropped. **

In [45]:
print('Cluster stores by sales correlation.')
sales_pivot  = pd.pivot_table(train,values='Sales',index='Date', columns=['Store'],aggfunc='mean').dropna()
print(sales_pivot.head())


Cluster stores by sales correlation.
Store         1       2       3        4       5       6       7       8     \
Date                                                                          
2013-01-02  5530.0  4422.0  6823.0   9941.0  4253.0  6089.0  8244.0  5419.0   
2013-01-03  4327.0  4159.0  5902.0   8247.0  3465.0  5398.0  7231.0  4842.0   
2013-01-04  4486.0  4484.0  6069.0   8290.0  4456.0  6092.0  7758.0  4059.0   
2013-01-05  4997.0  2342.0  4523.0  10338.0  1590.0  3872.0  5218.0  2337.0   
2013-01-06     0.0     0.0     0.0      0.0     0.0     0.0     0.0     0.0   

Store         9       10     ...      1106    1107    1108    1109    1110  \
Date                         ...                                             
2013-01-02  4903.0  4812.0   ...    5099.0  3955.0  6220.0  4576.0  4126.0   
2013-01-03  4602.0  4675.0   ...    4330.0  3151.0  4779.0  3654.0  3508.0   
2013-01-04  4798.0  5114.0   ...    3956.0  3990.0  5491.0  3596.0  3933.0   
2013-01-05  4254.0 

Correlation between sales of different stroes is then computed based on the sales pivot table.

Pearson's correlation coefficient is the covariance of the two variables divided by the product of their standard deviations.

In [46]:
sales_corr   = sales_pivot.corr()
sales_corr

Store,1,2,3,4,5,6,7,8,9,10,...,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115
Store,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.833043,0.882761,0.951319,0.792367,0.859146,0.870347,0.795846,0.892283,0.914628,...,0.851554,0.885964,0.769789,0.890809,0.867332,0.863381,0.905691,0.925063,0.935261,0.914890
2,0.833043,1.000000,0.943403,0.842329,0.954955,0.914589,0.925720,0.938276,0.854018,0.902346,...,0.947622,0.880210,0.950276,0.907091,0.935811,0.926877,0.872769,0.889416,0.823691,0.808041
3,0.882761,0.943403,1.000000,0.866312,0.926369,0.908120,0.934677,0.919314,0.897616,0.914340,...,0.934485,0.925251,0.922113,0.940136,0.932287,0.957384,0.929078,0.897752,0.843933,0.852999
4,0.951319,0.842329,0.866312,1.000000,0.794639,0.840521,0.900652,0.836020,0.916225,0.944953,...,0.872044,0.886220,0.780798,0.872804,0.898905,0.847424,0.870559,0.948669,0.966271,0.928537
5,0.792367,0.954955,0.926369,0.794639,1.000000,0.926716,0.924783,0.928777,0.823604,0.875888,...,0.942647,0.842644,0.966722,0.893254,0.922481,0.918472,0.860604,0.863275,0.778409,0.768393
6,0.859146,0.914589,0.908120,0.840521,0.926716,1.000000,0.906587,0.863868,0.790342,0.884659,...,0.912234,0.834767,0.896992,0.883091,0.883179,0.908468,0.889825,0.886765,0.827918,0.759295
7,0.870347,0.925720,0.934677,0.900652,0.924783,0.906587,1.000000,0.929505,0.897338,0.943562,...,0.959586,0.885001,0.909224,0.910047,0.939107,0.916510,0.874178,0.935976,0.899704,0.859055
8,0.795846,0.938276,0.919314,0.836020,0.928777,0.863868,0.929505,1.000000,0.880306,0.910330,...,0.950998,0.863519,0.948565,0.889421,0.954630,0.898791,0.840348,0.887643,0.819155,0.814436
9,0.892283,0.854018,0.897616,0.916225,0.823604,0.790342,0.897338,0.880306,1.000000,0.936246,...,0.885201,0.905997,0.827621,0.901169,0.914804,0.882752,0.855475,0.902078,0.910479,0.950183


**A pivot table is produced which shows the customer# value in all the different store for each given date.
Before jumping into the actual pivot table creation, NA values are dropped. **

In [47]:
print('Cluster stores by customer correlation.')
cust_pivot  = pd.pivot_table(train,values='Customers',index='Date', columns=['Store'],aggfunc='mean').dropna()
print(cust_pivot.head())

Cluster stores by customer correlation.
Store        1      2      3       4      5      6      7      8      9     \
Date                                                                         
2013-01-02  668.0  650.0  805.0  1429.0  577.0  781.0  955.0  698.0  481.0   
2013-01-03  578.0  555.0  721.0  1248.0  491.0  689.0  867.0  662.0  453.0   
2013-01-04  619.0  574.0  690.0  1232.0  533.0  724.0  870.0  517.0  497.0   
2013-01-05  635.0  324.0  525.0  1514.0  202.0  467.0  575.0  312.0  450.0   
2013-01-06    0.0    0.0    0.0     0.0    0.0    0.0    0.0    0.0    0.0   

Store        10    ...     1106   1107   1108   1109   1110   1111   1112  \
Date               ...                                                      
2013-01-02  521.0  ...    620.0  525.0  701.0  444.0  507.0  505.0  892.0   
2013-01-03  491.0  ...    550.0  461.0  604.0  392.0  491.0  444.0  809.0   
2013-01-04  517.0  ...    507.0  514.0  640.0  384.0  537.0  453.0  861.0   
2013-01-05  489.0  ...    36

Correlation between customer# of different stroes is then computed based on the sales pivot table.


In [48]:
cust_corr = cust_pivot.corr()
cust_corr

Store,1,2,3,4,5,6,7,8,9,10,...,1106,1107,1108,1109,1110,1111,1112,1113,1114,1115
Store,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.813300,0.889883,0.960181,0.767185,0.841527,0.848845,0.785128,0.900587,0.922888,...,0.851480,0.921708,0.755362,0.888946,0.881819,0.858699,0.925661,0.918122,0.922491,0.917729
2,0.813300,1.000000,0.914669,0.821572,0.940203,0.901667,0.897296,0.904356,0.842560,0.894769,...,0.917280,0.845297,0.914424,0.884097,0.899938,0.897457,0.844682,0.866056,0.797102,0.772251
3,0.889883,0.914669,1.000000,0.875303,0.918239,0.920370,0.934018,0.913708,0.912612,0.935955,...,0.944858,0.916162,0.914725,0.947677,0.935111,0.966123,0.926968,0.917530,0.843646,0.842927
4,0.960181,0.821572,0.875303,1.000000,0.773497,0.829555,0.878753,0.822348,0.925946,0.948948,...,0.869804,0.926848,0.768680,0.882822,0.914948,0.842724,0.908740,0.938413,0.955016,0.938947
5,0.767185,0.940203,0.918239,0.773497,1.000000,0.936342,0.910996,0.924003,0.818001,0.879172,...,0.938785,0.806293,0.966822,0.887094,0.905945,0.919970,0.826816,0.852274,0.744257,0.723582
6,0.841527,0.901667,0.920370,0.829555,0.936342,1.000000,0.909142,0.875835,0.800874,0.898846,...,0.927910,0.840464,0.924183,0.891013,0.894686,0.931968,0.883879,0.890770,0.801342,0.734714
7,0.848845,0.897296,0.934018,0.878753,0.910996,0.909142,1.000000,0.917360,0.889705,0.942134,...,0.959185,0.875272,0.902547,0.911785,0.929207,0.915789,0.866190,0.923895,0.882888,0.829351
8,0.785128,0.904356,0.913708,0.822348,0.924003,0.875835,0.917360,1.000000,0.874081,0.914476,...,0.954005,0.835465,0.937608,0.885132,0.944582,0.898213,0.825146,0.898562,0.791138,0.777610
9,0.900587,0.842560,0.912612,0.925946,0.818001,0.800874,0.889705,0.874081,1.000000,0.944959,...,0.885568,0.930938,0.813403,0.916989,0.924676,0.881381,0.893014,0.900526,0.904990,0.948717


For clustering we chose the distance measure as Pearson affinity since the feature values are correlation coefficients. We chose hierarchial clustering as the shape of the clusters might not be hyper-spherical and the distance of our choice is non-eucledian.

In [49]:
def pearson_affinity(M):
    return 1 - np.array([[pearsonr(a,b)[0] for a in M] for b in M])

In [50]:
cluster = AgglomerativeClustering(n_clusters=5, linkage='average',affinity=pearson_affinity).fit(sales_corr)
store['SalesCluster'] = cluster.labels_

Similary we cluster the customer# correlation values.

In [51]:
cust_corr   = cust_pivot.corr()
cluster = AgglomerativeClustering(n_clusters=5, linkage='average',affinity=pearson_affinity).fit(cust_corr)
store['CustomerCluster'] = cluster.labels_

In [53]:
# store.head()
print (store[store['SalesCluster']==4])

      Store StoreType Assortment  CompetitionDistance  \
84       85         b          a               1870.0   
258     259         b          b                210.0   
261     262         b          a               1180.0   
273     274         b          b               3640.0   
352     353         b          b                900.0   
422     423         b          a               1270.0   
511     512         b          b                590.0   
675     676         b          b               1410.0   
732     733         b          b                860.0   
947     948         b          b               1430.0   
1096   1097         b          b                720.0   

      CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
84                         10.0                    2011.0       0   
258                         NaN                       NaN       0   
261                         5.0                    2013.0       0   
273                         NaN        

Now the newly added cluster numbers can be used as a feature to futher predict future sales. Thus the whole point of the clustering operation was to engineer better features that bring out inherently hidden traits that can be used as good indicators to predict future sales patterns. 